## ICP project for HPPL 


- imports 

In [63]:
from math import *
import numpy as np
import os
from sys import argv
from random import shuffle
import cupy as cp
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import timeit

In [64]:
s = cp.array([1, -2, 3])
v = cp.array([4, 5, 6])
n = cp.append(s ,2.0)
w = n[-1]
m = cp.divide(n[:-1],w )
d = cp.cumsum(cp.linalg.norm(cp.subtract(s, v), ord=2))
l = cp.append(cp.cross(s, v),v).reshape(1,6)
ll = cp.transpose(l)
bi = cp.subtract(v, s) @ v
ll*bi

array([[-1755],
       [  390],
       [  845],
       [  260],
       [  325],
       [  390]])

In [65]:
class Point:
    def __init__(self, s=cp.array([]), n=cp.array([])):
        self.s = s.copy()
        self.n = n.copy()
        self.d = len(s)

    def copy(self):
        newS = self.s.copy()
        newN = self.n.copy()
        newP = Point(newS, newN)
        return newP

    def distTo(self, p):
        return sqrt(self.distSqdTo(p))

    def distSqdTo(self, p):
        distSqd = cp.cumsum(cp.linalg.norm(cp.subtract(self.s, p.s), ord=2))
        return distSqd

    def transform(self, m):
        old = self.s.copy()
        old = cp.append(old, 1.0)
        new = cp.dot(m , old)
        w = new[-1]
        self.s = cp.divide(new[:-1],w )
        return self


In [66]:
class Box:
    def __init__(self, pMin, pMax):
        self.sMin = pMin.s.copy()
        self.sMax = pMax.s.copy()
        self.d = pMin.d
        for i in range(len(self.sMin)):
            min = pMin.s[i]
            max = pMax.s[i]
            if (min > max):
                self.sMin[i] = max
                self.sMax[i] = min

    def copy(self):
        return Box(Point(self.sMin), Point(self.sMax))

    def updateMax(self, val, dim):
        if (val < self.sMin[dim]):
            return
        else:
            self.sMax[dim] = val

    def updateMin(self, val, dim):
        if (val > self.sMax[dim]):
            return
        else:
            self.sMin[dim] = val

    def range(self, dim):
        if (dim < 0 or dim >= self.d):
            return 0

        return self.sMax[dim] - self.sMin[dim]

    def intersects(self, that):
        for min, max in zip(that.sMin, self.sMax):
            if (max < min):
                return False
        for min, max in zip(self.sMin, that.sMax):
            if (max < min):
                return False

        return True

    def contains(self, p):
        for pval, max in zip(p.s, self.sMax):
            if (pval > max):
                return False
        for pval, min in zip(p.s, self.sMin):
            if (pval < min):
                return False
        for min, max in zip(self.sMin, p.sMax):
            if (max < min):
                return False
        return True

    def distTo(self, p):
        return sqrt(self.distSqdTo(p))


    def distSqdTo(self, p):
        distSqd = 0.0
        for pval, min, max in zip(p.s, self.sMin, self.sMax):
            dv = 0.0
            if (pval < min):
                dv = pval - min
            elif (pval > max):
                dv = pval - max
            distSqd += dv*dv

        return distSqd


In [67]:
class Node:

    def __init__(self, p, key, bounds):
        self.key = key

        if (type(p) == Point):
            self.p = p
        else:
            self.p = None
            
        self.bounds = bounds
        if (type(bounds) == Box):
            self.bounds = bounds
        else:
            self.bounds = None

        self.left = None
        self.right = None

In [68]:
class KdTree:
    # Constructor takes int k defining the number of dimensions
    def __init__(self, k=3):
        self.k = k
        self.size = 0
        self.root = None
        self.minP = [-inf for i in range(k)]
        self.maxP = [inf for i in range(k)]

    def insert(self, p):
        bounds = Box(Point(self.minP), Point(self.maxP))
        self.root = self.put(self.root, p, 0, bounds)


    def put(self, node, p, dim, bounds):

        if (node == None):
            self.size += 1
            key = p.s[dim]
            return Node(p, key, bounds)
 
        cmp = p.s[dim] - node.key
        if (cmp < 0):
            upperBounds = bounds.copy()
            upperBounds.updateMax(node.key, dim)
            node.left = self.put(node.left, p, (dim + 1) % self.k, upperBounds)
        else:
            lowerBounds = bounds.copy()
            lowerBounds.updateMin(node.key, dim)
            node.right = self.put(node.right, p, (dim + 1) % self.k, lowerBounds)

        return node

    def nearest(self, p):
        root = self.root
        nearest_p, dist = self.find_nearest(root, p, 0, root.p, p.distSqdTo(root.p))
        return nearest_p

    def find_nearest(self, node, p, dim, candidate, dist):
        if (node == None):
            return candidate, dist

        if (dist < node.bounds.distSqdTo(p)):
            return candidate, dist

        newDist = p.distSqdTo(node.p)
        if (newDist < dist):
            candidate = node.p
            dist = newDist

        go_left = (p.s[dim] < node.key)

        for i in range(2):
            if (go_left):
                candidate, dist = self.find_nearest(node.left, p, (dim + 1) % self.k, candidate, dist)
            else:
                candidate, dist = self.find_nearest(node.right, p, (dim + 1) % self.k, candidate, dist)

            go_left = not go_left

        return candidate, dist

In [69]:
def load_xf(file_name):
    with open(file_name) as f:
        data = f.read()
        rows = []
        for r in data.split('\n'):
            if (len(r) > 0):
                rows.append(r)

        if (len(rows) != 4):
            rows = ["0 0 0 0" for i in range(4)]
        result = []
        for r in rows:
            c = []
            for v in r.split(' '):
                if (len(v) > 0):
                    c.append(float(v))
            if (len(c) != 4):
                c = [0, 0, 0, 0]
            result.append(c)
    return cp.asarray(result)

In [70]:
def load_pts(file_name):
    with open(file_name) as f:
        data = f.read()
        rows = data.split('\n')
        result = []
        for r in rows:
            if (len(r) == 0):
                continue
            pData = [float(v) for v in r.split(' ')]
            if (len(pData) != 6):
                pData = [0, 0, 0, 0, 0, 0]
            result.append(Point(cp.array(pData[0:3]), cp.array(pData[3:6])))
    return result


In [71]:
def write_xf(file_name, M):
    dir = os.path.dirname(file_name)
    if (not os.path.exists(dir)):
        os.makedirs(dir)

    with open(file_name, "w") as f:
        for row in M.A:
            for val in row:
                f.write(str(val))
                f.write(' ')
            f.write('\n')
    return


In [72]:
def write_pts(file_name, pts):
    dir = os.path.dirname(file_name)
    if (not os.path.exists(dir)):
        os.makedirs(dir)
    with open(file_name, "w") as f:
        for p in pts:
            for val in (p.s + p.n):
                f.write(str(val))
                f.write(' ')
            f.write('\n')
    return


In [73]:
from pathlib import Path

file1 = Path("./bunny/bun045.pts")
file2 = Path("./bunny/bun000.pts")


pts1 = load_pts(file1)
pts2 = load_pts(file2)


file1_xf = str(file1.with_suffix('')) + '.xf'
file2_xf = str(file2.with_suffix('')) + '.xf'



if (not Path(file1_xf).is_file()):
    M1 = np.identity(4)
else:
    M1 = load_xf(file1_xf)
    
output_file2_xf = './output/' + Path(file2_xf).name + '.xf'

if (not Path(output_file2_xf).is_file()):
    if (not Path(file2_xf).is_file()):
        M2 = np.identity(4)
    else:
        M2 = load_xf(file2_xf)
else:
    M2 = load_xf(output_file2_xf)

kdtree = KdTree()
for p in pts2:
    kdtree.insert(p)

# ICP iteration (until improvement is less than 0.01%)
print("Starting iteration...")
ratio = 0.0
M2_inverse = cp.linalg.inv(M2)
pts_index = list(range(len(pts1)))
count = 0
while (ratio < 0.9999):
    shuffle(pts_index)
    # Apply M1 and the inverse of M2
    p = [pts1[i].copy().transform(M1).transform(M2_inverse) for i in pts_index[:1000]]
    q = [kdtree.nearest(point) for point in p]


    # Compute point to plane distances
    point2plane = cp.array([ cp.abs(cp.subtract(pi.s, qi.s) @ qi.n) for pi,qi in zip(p,q)])
    median_3x = 3.0 * cp.median(point2plane)

    # Cull outliers
    point_pairs = []
    dist_sum = 0.0
    for i, pair in enumerate(zip(p,q)):
        if (point2plane[i] <= median_3x):
            point_pairs.append(pair)
            dist_sum += point2plane[i]
    if (len(point_pairs) > 0):
        old_mean = dist_sum/len(point_pairs)
    else:
        print("Error: Something went wrong when computing distance means")
        quit()

    C = cp.zeros(shape=(6,6))
    d = cp.zeros(shape=(6,1))
    for (p, q) in point_pairs:
        Ai = cp.append(cp.cross(p.s, q.n),q.n).reshape(1,6)
        AiT = cp.transpose(Ai)
        bi = cp.subtract(q.s, p.s) @ q.n

        C += AiT*Ai
        d += AiT*bi
    x = cp.linalg.solve(C,d).flatten()
    rx,ry,rz,tx,ty,tz = x
    Micp = cp.asarray([[1.0, ry*rx - rz, rz*rx + ry, tx],
                    [rz, 1.0 + rz*ry*rx, rz*ry - rx, ty], 
                    [-ry, rx, 1.0, tz], 
                    [0, 0, 0, 1.0]])

    # Compute new mean point-to-plane distance
    dist_sum = 0.0
    for (p, q) in point_pairs:
        # Apply Micp
        p = p.transform(Micp)

        dist_sum += cp.abs( cp.subtract(p.s, q.s)@ q.n)
    new_mean = dist_sum/len(point_pairs)
    count += 1
    ratio = new_mean / old_mean

    # Update M1 iff we improved (otherwise, but NOT only then, we will terminate)
    if (ratio < 1.0):
        M1 = M2*Micp*M2_inverse*M1
    else:
        new_mean = old_mean

    print("Finished iteration #{} with improvement of {:2.4%}".format(count, 1.0 - ratio))

print("Terminated successfully with a sampled mean distance of {}".format(new_mean))

# Write results to file
output_file1_pts = './output/' + str(file1).split('/')[-1]
output_file1_xf = './output/' + file1_xf.split('/')[-1]
output_file2_pts = './output/' + str(file2).split('/')[-1]
output_file2_xf = './output/' + file2_xf.split('/')[-1]
write_pts(output_file1_pts, pts1)
write_pts(output_file2_pts, pts2)
write_xf(output_file1_xf, M1)
write_xf(output_file2_xf, M2)

print ('Done!!!')


Starting iteration...


KeyboardInterrupt: 

In [ ]:
a = [1, 2, 3, 4]
b = [3, 4, 5, 6]
c=cp.subtract(cp.array(a),cp.array(b)) 
c 

array([-2, -2, -2, -2])

In [ ]:
a = cp.array([1, 2, 3, 4])
a.reshape(4, 1)

array([[1],
       [2],
       [3],
       [4]])

In [74]:
# ICP iteration (until improvement is less than 0.01%)
print("Starting iteration...")
ratio = 0.0
M2_inverse = cp.linalg.inv(M2)
pts_index = list(range(len(pts1)))
count = 0
while (ratio < 0.9999):
    shuffle(pts_index)
    # Apply M1 and the inverse of M2
    p = [pts1[i].copy().transform(M1).transform(M2_inverse) for i in pts_index[:1000]]
    q = [kdtree.nearest(point) for point in p]


    # Compute point to plane distances
    point2plane = cp.array([ cp.abs(cp.subtract(pi.s, qi.s) @ qi.n) for pi,qi in zip(p,q)])
    median_3x = 3.0 * cp.median(point2plane)

    # Cull outliers
    point_pairs = []
    dist_sum = 0.0
    for i, pair in enumerate(zip(p,q)):
        if (point2plane[i] <= median_3x):
            point_pairs.append(pair)
            dist_sum += point2plane[i]
    if (len(point_pairs) > 0):
        old_mean = dist_sum/len(point_pairs)
    else:
        print("Error: Something went wrong when computing distance means")
        quit()

    C = cp.zeros(shape=(6,6))
    d = cp.zeros(shape=(6,1))
    for (p, q) in point_pairs:
        Ai = cp.append(cp.cross(p.s, q.n),q.n).reshape(1,6)
        AiT = cp.transpose(Ai)
        bi = cp.subtract(q.s, p.s) @ q.n

        C += AiT*Ai
        d += AiT*bi
    x = cp.linalg.solve(C,d).flatten()
    rx,ry,rz,tx,ty,tz = x
    Micp = cp.asarray([[1.0, ry*rx - rz, rz*rx + ry, tx],
                    [rz, 1.0 + rz*ry*rx, rz*ry - rx, ty], 
                    [-ry, rx, 1.0, tz], 
                    [0, 0, 0, 1.0]])

    # Compute new mean point-to-plane distance
    dist_sum = 0.0
    for (p, q) in point_pairs:
        # Apply Micp
        p = p.transform(Micp)

        dist_sum += cp.abs( cp.subtract(p.s, q.s)@ q.n)
    new_mean = dist_sum/len(point_pairs)
    count += 1
    ratio = new_mean / old_mean

    # Update M1 iff we improved (otherwise, but NOT only then, we will terminate)
    if (ratio < 1.0):
        M1 = M2*Micp*M2_inverse*M1
    else:
        new_mean = old_mean

    print("Finished iteration #{} with improvement of {:2.4%}".format(count, 1.0 - ratio))

print("Terminated successfully with a sampled mean distance of {}".format(new_mean))

# Write results to file
output_file1_pts = './output/' + str(file1).split('/')[-1]
output_file1_xf = './output/' + file1_xf.split('/')[-1]
output_file2_pts = './output/' + str(file2).split('/')[-1]
output_file2_xf = './output/' + file2_xf.split('/')[-1]
write_pts(output_file1_pts, pts1)
write_pts(output_file2_pts, pts2)
write_xf(output_file1_xf, M1)
write_xf(output_file2_xf, M2)

print ('Done!!!')

Starting iteration...


KeyboardInterrupt: 